**City - Dummy + STE_Counter_Graph\
POIs = 6**

In [2]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

In [3]:
# Load data
cost_data = pd.read_excel('/Users/priyanshukumarjha/Desktop/Itinerary Data/Trial _Sheet_Data/Trial_Sheet_15/buda_cost.xlsx')
utility_buda = pd.read_excel('/Users/priyanshukumarjha/Desktop/Itinerary Data/Trial _Sheet_Data/Trial_Sheet_15/hehe.xlsx')

In [4]:
cost_data

,from,to,cost
0,1,2,2667
1,1,3,12000
2,1,4,12000
3,1,5,12000
4,1,6,12000
5,2,1,2667
6,2,3,12000
7,2,4,12000
8,2,5,12000
9,2,6,2667


In [5]:
utility_buda

,poiID,poiName,theme,Avg Visiting TIme,Utility Score,opening time,closing time,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,1,A,Structure,0,0,00:00:00,23:59:00,1,1,1,1,1,1,1
1,2,B,Structure,10,20,00:00:00,23:59:00,1,1,1,1,1,1,1
2,3,C,Structure,10,20,00:00:00,23:59:00,1,1,1,1,1,1,1
3,4,D,Precinct,10,20,00:00:00,23:59:00,1,1,1,1,1,1,1
4,5,E,Hotel,10,20,00:00:00,23:59:00,1,1,1,1,1,1,1
5,6,F,Library,0,0,00:00:00,23:59:00,1,1,1,1,1,1,1


In [6]:
# Extract relevant data
poi_ids = utility_buda['poiID'].tolist()
visit_times = dict(zip(utility_buda['poiID'], utility_buda['Avg Visiting TIme']))
utility_scores = dict(zip(utility_buda['poiID'], utility_buda['Utility Score']))

In [7]:
poi_ids

[1, 2, 3, 4, 5, 6]

In [8]:
visit_times

{1: 0, 2: 10, 3: 10, 4: 10, 5: 10, 6: 0}

In [9]:
utility_scores

{1: 0, 2: 20, 3: 20, 4: 20, 5: 20, 6: 0}

In [10]:
# Create a dictionary for travel times in minutes (cost in meters converted to km then multiplied by 15)
travel_times = {(row['from'], row['to']): (row['cost'] / 1000) * 15 for _, row in cost_data.iterrows()}

In [11]:
# Extract opening and closing times into dictionaries
opening_times = utility_buda.set_index("poiID")["opening time"].to_dict()
closing_times = utility_buda.set_index("poiID")["closing time"].to_dict()

In [12]:
closing_times

{1: datetime.time(23, 59),
 2: datetime.time(23, 59),
 3: datetime.time(23, 59),
 4: datetime.time(23, 59),
 5: datetime.time(23, 59),
 6: datetime.time(23, 59)}

In [13]:
travel_times

{(1, 2): 40.004999999999995,
 (1, 3): 180.0,
 (1, 4): 180.0,
 (1, 5): 180.0,
 (1, 6): 180.0,
 (2, 1): 40.004999999999995,
 (2, 3): 180.0,
 (2, 4): 180.0,
 (2, 5): 180.0,
 (2, 6): 40.004999999999995,
 (3, 1): 180.0,
 (3, 2): 180.0,
 (3, 4): 40.004999999999995,
 (3, 5): 40.004999999999995,
 (3, 6): 180.0,
 (4, 1): 180.0,
 (4, 2): 180.0,
 (4, 3): 40.004999999999995,
 (4, 5): 40.004999999999995,
 (4, 6): 180.0,
 (5, 1): 180.0,
 (5, 2): 180.0,
 (5, 3): 40.004999999999995,
 (5, 4): 40.004999999999995,
 (5, 6): 180.0,
 (6, 1): 180.0,
 (6, 2): 40.004999999999995,
 (6, 3): 180.0,
 (6, 4): 180.0,
 (6, 5): 180.0}

## TIME BUDGET

In [15]:
# Time budget in minutes 
time_budget = 400

In [16]:
model = Model("ILP_Model_1")

Set parameter Username
Set parameter LicenseID to value 2616933
Academic license - for non-commercial use only - expires 2026-01-31


## DECLARING VARIABLES

In [18]:
# Decision variables: y[i] = 1 if POI i is included in the itinerary, 0 otherwise
y = model.addVars(poi_ids, vtype=GRB.BINARY, name="y")

In [19]:
# Introduce new binary variables for travel between POIs
z = model.addVars(poi_ids, poi_ids, vtype=GRB.BINARY, name="z")

In [20]:
start_time = model.addVar(vtype=GRB.CONTINUOUS, name="start_time")

In [21]:
arrival_time = model.addVars(poi_ids, vtype=GRB.CONTINUOUS, name="arrival_time")

In [22]:
N = len(poi_ids)  # Total number of POIs

# # Create continuous variables for the position of each POI in the sequence
# p = model.addVars(poi_ids, vtype=GRB.CONTINUOUS, lb=2, ub=N, name="p")

## OBJECTIVE FUNCTION

In [24]:
# Objective: Maximize the sum of utility scores for selected POIs
model.setObjective(quicksum(utility_scores[i] * y[i] for i in poi_ids), GRB.MAXIMIZE)

## TIME CONSTRAINT

In [26]:
# Ensure total travel + visit time does not exceed the time budget
model.addConstr(
    quicksum(travel_times.get((i, j), 0) * z[i, j] for i in poi_ids for j in poi_ids if i != j) +
    quicksum(visit_times[i] * y[i] for i in poi_ids) <= time_budget,
    name="TimeConstraint"
)

<gurobi.Constr *Awaiting Model Update*>

## CATEGORY CONSTRAINT (MADE BOUNDS EMPTY TO AVOID THIS CONSTRAINT)

In [28]:
# Define lower and upper bounds for each theme
theme_bounds = {
    
}

# Add constraints for each theme
for theme, (lower_bound, upper_bound) in theme_bounds.items():
    # Sum up the binary variables y[i] for all POIs belonging to the current theme
    theme_count = quicksum(y[i] for i in poi_ids if i - 1 < len(utility_buda) and utility_buda.iloc[i - 1]["theme"] == theme)

    
    # Add lower and upper bound constraints
    model.addConstr(theme_count >= lower_bound, name=f"Min_{theme}")
    model.addConstr(theme_count <= upper_bound, name=f"Max_{theme}")


## ORDEREING CONSTRAINT (MADE LIST EMPTY TO AVOID THIS CONSTRAINT)

In [30]:
# List of ordering constraints in the form of (a, b)
ordering_constraints = []  

In [31]:
def convert_to_minutes(time_obj):
    return time_obj.hour * 60 + time_obj.minute

# Calculate T_ij dynamically
for (a, b) in ordering_constraints:
    closing_time_a = convert_to_minutes(closing_times[a])  # Convert closing time to minutes
    #T_ij = closing_time_a + visit_times[a] + travel_times.get((a, b), 0)
    T_ij=10000
    
    model.addConstr(
        arrival_time[a] + visit_times[a] + travel_times.get((a, b), 0) * y[a] <= arrival_time[b] + T_ij * (1 - y[b]),
        name=f"Ordering_{a}_before_{b}"
    )

## Z[i,i] = 0

In [33]:
# z[i, i] = 0, for i = 1..N
model.addConstrs(
    (z[i, i] == 0 for i in range(1, N+1)),
    name="diagonal_zero"
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>}

## LOGICAL RELATIONSHIP CONSTRAINT BETWEEN Y AND Z

In [35]:
# Additional constraint: Ensure logical connection between y[i] and z[i, j]
for i in poi_ids:
    for j in poi_ids:
        if i != j:
            model.addConstr(z[i, j] <= y[i], name=f"TravelStartsFrom_{i}_{j}")
            model.addConstr(z[i, j] <= y[j], name=f"TravelEndsAt_{i}_{j}")

In [36]:
# y_i <= sum_{j=1..N} z_{i,j}, for i = 1..N-1
model.addConstrs(
    (y[i] <= quicksum(z[i, j] for j in range(1, N+1)))
    for i in range(1, N)
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [37]:
# y_j <= sum_{i=1..N} z_{i,j}, for j = 2..N
model.addConstrs(
    (y[j] <= quicksum(z[i, j] for i in range(1, N+1)))
    for j in range(2, N+1)
)

{2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>}

In [38]:
for i in poi_ids:
    print(y[i])

<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>


In [39]:
for i in poi_ids:
    for j in poi_ids:
        if i!=j:
            print(z[i,j])

<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<

In [40]:
starting_poi = poi_ids[0]
ending_poi = poi_ids[-1]

In [41]:
ending_poi

6

## POI 1 AND N SHOULD ALWAYS BE PART OF ITINERARY

In [43]:
# THIS CONSTRAINT IS ALREADY IMPOSED BY OTHER CONSTRAINTS

# # Constraint: POI 1 must always be part of the itinerary (start point)
# model.addConstr(y[starting_poi] == 1, name="StartPoint")

# # Constraint: POI 8 must always be part of the itinerary (end point)
# model.addConstr(y[ending_poi] == 1, name="EndPoint")

In [44]:
# Start point: Exactly one outgoing edge from node 1
model.addConstr(quicksum(z[starting_poi, j] for j in poi_ids if j != starting_poi) == 1, name="StartConstraint")

# End point: Exactly one incoming edge to node 6
model.addConstr(quicksum(z[i, ending_poi] for i in poi_ids if i != ending_poi) == 1, name="EndConstraint")

<gurobi.Constr *Awaiting Model Update*>

## CONNECTIVITY CONSTRAINT

In [46]:
# Connectivity: Each intermediate POI must have exactly one incoming and one outgoing edge if it is selected
for k in poi_ids:
    if k not in [starting_poi, ending_poi]:  # Skip start and end points
        model.addConstr(quicksum(z[i, k] for i in poi_ids if i != k) == y[k], name=f"FlowIn_{k}")
        model.addConstr(quicksum(z[k, j] for j in poi_ids if j != k) == y[k], name=f"FlowOut_{k}")

## SUBTOUR ELIMINATION

In [48]:
# # Add the subtour elimination constraints
# for i in poi_ids:
#     for j in poi_ids:
#         if i != j and i > starting_poi and j > starting_poi:  # Skip the start POI
#             model.addConstr(
#                 p[i] - p[j] + 1 <= (N - 1) * (1 - z[i, j]),
#                 name=f"SubtourElimination_{i}_{j}"
#             )

## No incoming edges from any POI to starting_poi

In [50]:
# No outgoing edges from POI ending_poi
model.addConstr(quicksum(z[i, starting_poi] for i in poi_ids if i != starting_poi) == 0, name="NoIncomingToStart")

<gurobi.Constr *Awaiting Model Update*>

## No outgoing edges to any POI from ending_poi

In [52]:
# No outgoing edges from POI ending_poi
model.addConstr(quicksum(z[ending_poi, j] for j in poi_ids if j != ending_poi) == 0, name="NoOutgoingFromEnd")

<gurobi.Constr *Awaiting Model Update*>

## CONVERTING ALL OPENING AND CLOSING TIMES IN MINUTE FORMAT FROM 00:00AM FOR FURTHER CALCULATIONS

In [54]:
def convert_to_minutes(time_obj):
    return time_obj.hour * 60 + time_obj.minute

opening_times = {i: convert_to_minutes(t) for i, t in opening_times.items()}
closing_times = {i: convert_to_minutes(t) for i, t in closing_times.items()}


In [55]:
opening_times

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

## STARTING TIME SET TO 10AM (600 MINUTES FROM 00:00 AM)

In [57]:
#start_time is declared as a continuous variable, meaning it can take any real value between 420 (7:00 AM) and 600 (10:00 AM).
#The solver will treat start_time as a decision variable and explore all possible values in that range to find the optimal one that satisfies the objective and other constraints.

model.addConstr(start_time == 600, name="StartAfter10AM")

# model.addConstr(start_time >= 480, name="StartAfter8AM")  # 480 minutes = 8 AM
# model.addConstr(start_time <= 600, name="StartBefore10AM")  # 600 minutes = 10 AM

<gurobi.Constr *Awaiting Model Update*>

In [58]:
model.addConstr(arrival_time[starting_poi] == start_time, name="StartTimeAtSource")

<gurobi.Constr *Awaiting Model Update*>

In [59]:
for i in poi_ids:
    for j in poi_ids:
        if i != j:
            model.addConstr(
                arrival_time[i] >= (arrival_time[j] + visit_times[j] + travel_times.get((j, i), 0)) * z[j, i],
                name=f"ArrivalTime_{j}_to_{i}"
            )

In [60]:
for i in poi_ids:
    print(arrival_time[i])

<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>


## OPENING CLOSING TIME OF POI CONSTRAINT (NOT IN USE HERE AS ALL ARE OPEN THROUGHOUT THE DAY 00:00 AM TO 23:59 PM)

In [62]:
for i in poi_ids:
    model.addConstr(arrival_time[i] >= opening_times[i], name=f"OpeningTime_{i}")
    model.addConstr(arrival_time[i] <= closing_times[i] - visit_times[i], name=f"ClosingTime_{i}")

In [63]:
# Create a dictionary for day availability
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_availability = {day: {} for day in days_of_week}

# Populate the dictionary
for i, row in utility_buda.iterrows():
    poi_id = row['poiID']
    for day in days_of_week:
        day_availability[day][poi_id] = row[day]

## OPENINNG CLOSING DAY CONSTRAINT (NOT IN USE HERE AS ALL POIs ARE OPEN ON ALL THE 7 DAYS)

In [65]:
# Suppose the user selects a day
selected_day = 'Tuesday'

# Add constraints to ensure POIs are only selected if they are open on the chosen day
for i in poi_ids:
    if day_availability[selected_day][i] == 0:  # POI is closed on the selected day
        model.addConstr(y[i] == 0, name=f"Closed_POI_{i}_on_{selected_day}")

In [66]:
if str(i) in day_availability[selected_day]:
    if day_availability[selected_day][str(i)] == 0:
        model.addConstr(y[i] == 0, name=f"Closed_POI_{i}_on_{selected_day}")

In [67]:
print(day_availability[selected_day].keys())

dict_keys([1, 2, 3, 4, 5, 6])


In [68]:
# model.addConstr(z[3, 4] == 1, name="fix_z_3_4")
# model.addConstr(z[4, 5] == 1, name="fix_z_4_5")
# model.addConstr(z[5, 3] == 1, name="fix_z_5_3")

In [69]:
# def my_callback(model, where):
#     # Check if a new integer solution is found
#     if where == GRB.Callback.MIPSOL:
#         # Get the current objective value
#         obj_val = model.cbGet(GRB.Callback.MIPSOL_OBJ)
#         print("New incumbent solution found with objective value:", obj_val)
#         # Print the value of each variable
#         for v in model._vars:
#             val = model.cbGetSolution(v)
#             print(f"  {v.VarName} = {val}")
#         print("-------------------------------")

In [70]:
# # Store references to variables in model object so the callback can see them
# model._vars = model.getVars()

In [71]:
# # Optimize with callback
# model.optimize(my_callback)

In [72]:
# Optimize with callback
model.optimize()

# Check if the model is infeasible
if model.status == GRB.INFEASIBLE:
    print("The model is infeasible. Computing IIS...")
    
    # Compute the IIS
    model.computeIIS()
    
    # Write the IIS to a file for a complete report (optional)
    model.write("model.ilp")
    
    # Iterate over constraints to print those that are in the IIS
    print("The following constraints are in the IIS:")
    for constr in model.getConstrs():
        if constr.IISConstr:
            print(f"  {constr.ConstrName}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.0.0 24A335)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 103 rows, 49 columns and 313 nonzeros
Model fingerprint: 0x889b555a
Model has 30 quadratic constraints
Variable types: 7 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+02]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 89 rows and 27 columns
Presolve time: 0.00s
Presolved: 44 rows, 52 columns, 137 nonzeros
Presolved model has 20 SOS constraint(s)
Variable types: 24 continuous, 28 integer (27 binary)
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 20.0000000

Root relaxation: objective 8.000000e+01, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current N

In [73]:
selected_edges = [(i, j) for i in poi_ids for j in poi_ids if i != j and z[i, j].X > 0.0]

In [74]:
selected_edges

[(1, 2), (2, 6)]

In [75]:
if model.status == GRB.OPTIMAL:
    # Calculate the LHS of the time constraint after the solution
    total_travel_time = sum(travel_times.get((i, j), 0) * z[i, j].X for i in poi_ids for j in poi_ids if i != j)
    total_visit_time = sum(visit_times[i] * y[i].X for i in poi_ids)
    total_time_taken = total_travel_time + total_visit_time
    
    print(f"Total Travel Time: {total_travel_time:.2f} minutes")
    print(f"Total Visit Time: {total_visit_time:.2f} minutes")
    print(f"Total Time Taken: {total_time_taken:.2f} minutes")
else:
    print("No optimal solution found.")

Total Travel Time: 80.01 minutes
Total Visit Time: 10.00 minutes
Total Time Taken: 90.01 minutes


In [76]:
# Extract and print the optimized utility score and the selected POIs
if model.status == GRB.OPTIMAL:
    optimized_utility_score = model.objVal
    selected_pois = [i for i in poi_ids if y[i].X > 0.5]
    print(f"Optimized Utility Score: {optimized_utility_score}")
    print(f"Selected POIs: {selected_pois}")
else:
    print("No optimal solution found.")

Optimized Utility Score: 20.0
Selected POIs: [1, 2, 6]
